In [2]:
!pip install pyspark
!pip install findspark

In [4]:
import os
import sys
import pyspark
import findspark
findspark.init()
findspark.find()

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("MySpark_App") \
       .getOrCreate()

spark

In [ ]:
# Read raw csv file from GitHub by converting the following steps:
# 1. Point the URL to the CSV file in raw format
# 2. Read the CSV directly into a pandas DataFrame
import pandas as pd

url = "https://raw.githubusercontent.com/RajashekarAllala/Python_For_Data_Engineering/refs/heads/main/Data_Sets/employee_data.csv"

# 2. Read the CSV directly into a pandas DataFrame
get_csv = pd.read_csv(url)

# 3. Convert the pandas DataFrame to a Spark DataFrame
df_csv = spark.createDataFrame(get_csv)

# 4. Show the first few rows to verify
display(df_csv)

In [ ]:
# Read data from a table available in catalog
df_table = spark.read.table("data_sets.organization.employee_data")
df_table.show(100)

In [ ]:
# Drop tax_file_no as it contains NULL data
df_emp = df_table.drop("tax_file_no")
df_emp.show(20)

In [ ]:
# Check schema
print("Schema of df_emp")
df_emp.printSchema()

# Check data types
print("Data types of df_emp")
df_emp.dtypes


In [ ]:
# df_emp: Change data types of number to int, and annual_salary to float.
# Add all in one statement to optimize performance.
df_emp = df_emp.withColumn("number", F.col("number").cast(T.IntegerType())) \
               .withColumn("annual_salary", F.col("annual_salary").cast(T.FloatType()))
df_emp.printSchema()


In [ ]:
df_emp.show()


In [ ]:
# Check if there are any null values in gender, employment_status, annual_salary only
df_emp.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_emp.columns]).show()


In [ ]:
# Handle null values in gender, employment_status, annual_salary only
# df_emp: Replace null values in gender with 'Unknown', employment_status with 'Unknown', annual_salary with 0.0
df_emp = df_emp.fillna({'gender': 'Unknown', 'employment_status': 'Unknown', 'annual_salary': 0.0})
df_emp.show()
df_emp.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_emp.columns]).show()

In [ ]:
# Handle null values in birth_date
# df_emp: Replace null values in birth_date with '1900-01-01'
df_emp = df_emp.fillna({'birth_date': '1900-01-01'})
df_emp.show()
df_emp.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_emp.columns]).show()

In [ ]:
# Decide to drop or fill the values with NULL in first and last names
# df_emp: Drop rows with NULL values in first and last names
df_emp = df_emp.dropna(subset=['first_name', 'last_name'])
df_emp.show()
df_emp.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_emp.columns]).show()

In [ ]:
# Find Average, Max, and Min Salary using select
df_emp.select(F.avg('annual_salary'), F.max('annual_salary'), F.min('annual_salary')).show()

In [ ]:
# Using groupBy employment_status find avg, max, and min
df_emp.groupBy('employment_status').agg(F.avg('annual_salary'), F.max('annual_salary'), F.min('annual_salary')).orderBy("employment_status").show()

In [ ]:
# Find number of employees in each employment_status
df_emp.groupBy('employment_status').count().show()

In [ ]:
# Find employees count based on gender
df_emp.groupBy('gender').count().orderBy('gender').show()

df_emp.groupBy("gender", "employment_status").count().orderBy('gender').show()

